## 2.1.3 Stereo Calibration w/pixelization
Stereo calibration and triangulation for 20 meters

### import  libraries

In [1]:
vscode = 1

In [2]:
if(vscode == 1):
    # for vscode
    %matplotlib qt
else:
    # for jupyter notebook
    from mpl_toolkits.mplot3d import axes3d
    import matplotlib.pyplot as plt

    %matplotlib notebook


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 
from typing import Sequence
from calib_lib import *
DECIMALS = 2  # how many decimal places to use in print


In [4]:
F = 16                                       # focal length( in mm )
image_size = np.array([1936,1216])               # sensor size(in mm)
pixel_width = 5.86e-3                       # pixel size in mm
PX= image_size[0]/2.0                       # principal point x-coordinate
PY= image_size[1]/2.0                       # principal point y-coordinate
IMAGE_HEIGTH = image_size[1]
IMAGE_WIDTH = image_size[0]
THETA_X = 0                                 # roll angle
THETA_Y = 0                                 # pitch angle
THETA_Z = 0                                 # yaw angle


# camera Right
THETA_X_R = 0                                 # roll angle
THETA_Y_R = 0                                 # pitch angle
THETA_Z_R= 0                                 # yaw angle
# camera Left
THETA_X_L = 0                                 # roll angle
THETA_Y_L = 0                                 # pitch angle
THETA_Z_L= 0                                 # yaw angle

C_L = np.array([0,0,0])                     # camera centre

C_R = np.array([500,0,0])

chess_dimx,chess_dimy = (13,9)
chess_sq_size = 44

In [5]:
def repeat_func(xrange,yrange,zrange):

    translst= []

    for z in zrange:
        for x in xrange:
            for y in yrange:
                translst.append([x,y,z])
                
    len(translst)
    #print(translst)

    # chessboard print
    chess_pts = []
    world_pts = create_chessboard(chess_dimx,chess_dimy,chess_sq_size)
    rotangles = [[0,0,0],[np.pi/4,0,np.pi/2],[0,np.pi/4,np.pi/2],[0,np.pi/4,-np.pi/2],[np.pi/4,0,-np.pi/2]]

    for i in range(len(translst)):
        for j in range(len(rotangles)):    
            chess_pts.append(get_chessboard_rot_trans(world_pts,rotangles[j][0],rotangles[j][1],rotangles[j][2],translst[i][0],translst[i][1],translst[i][2]))

    chess_pts_arr = np.array(chess_pts)
    print(chess_pts_arr.shape)

    pattern_size_x,pattern_size_y = (chess_dimx,chess_dimy)

    X_pattern = np.linspace(0, pattern_size_x,pattern_size_x + 1)*chess_sq_size
    Y_pattern = np.linspace(0, pattern_size_y,pattern_size_y + 1)*chess_sq_size
    zdata = np.zeros((pattern_size_x + 1,pattern_size_y + 1))
    xdata, ydata = np.meshgrid(X_pattern, Y_pattern)

    xdata_ = xdata.flatten()
    ydata_ = ydata.flatten()
    zdata_ = zdata.flatten()
    
    # homogeneous coordinates
    world_pts =([xdata_,ydata_,zdata_])
    world_pts_ = np.array(world_pts).T

    world_pts_arr = np.zeros((chess_pts_arr.shape[0],(pattern_size_x+1)*(pattern_size_y+1),3),np.float32)
    
    for i in range(chess_pts_arr.shape[0]):
        world_pts_arr[i,:,:] = world_pts_

    world_pts_arr = world_pts_arr.reshape(world_pts_arr.shape[0],1,(pattern_size_x+1)*(pattern_size_y+1),3)
    
    x_lst_R = []
    x_lst_L = []
    for i in range(chess_pts_arr.shape[0]):
        x_arr_R, X_arr_R, E_R, K_R, P_R = get_image_points(chess_pts_arr[i,:,:],PX,PY,thetax= 0,thetay = 0,thetaz = 0,trans_x= -C_R[0],trans_y= -C_R[1],trans_z= -C_R[2],F = F,mx = (1/pixel_width),my =(1/pixel_width))
        x_arr_L, X_arr_L, E_L, K_L, P_L = get_image_points(chess_pts_arr[i,:,:],PX,PY,thetax= 0,thetay = 0,thetaz = 0,trans_x= -C_L[0],trans_y= -C_L[1],trans_z= -C_L[2],F = F,mx = (1/pixel_width),my =(1/pixel_width))
        
        x_lst_R.append(x_arr_R)
        x_lst_L.append(x_arr_L)

    x_zhang_R = np.array(x_lst_R,np.float32)
    x_zhang_L = np.array(x_lst_L,np.float32)

    print("World points: ",world_pts_arr.shape)
    print("Image points: ",x_zhang_L.shape)


    return world_pts_arr,x_zhang_R,x_zhang_L,P_R,P_L


In [6]:
def calib(world_pts_arr,x_zhang_R,x_zhang_L):    
    ret_R, mtx_R, dist_R, rvecs_R, tvecs_R = cv2.calibrateCamera(world_pts_arr, x_zhang_R, (image_size[0],image_size[1]), None, None)
    ret_L, mtx_L, dist_L, rvecs_L, tvecs_L = cv2.calibrateCamera(world_pts_arr, x_zhang_L, (image_size[0],image_size[1]), None, None)

    flags = 0
    flags |= cv2.CALIB_FIX_INTRINSIC
    # Here we fix the intrinsic camara matrixes so that only Rot, Trns, Emat and Fmat are calculated.
    # Hence intrinsic parameters are the same 

    criteria_stereo= (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


    # This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
    retS, new_mtxL, distL, new_mtxR, distR, Rot, Trns, Emat, Fmat = cv2.stereoCalibrate(world_pts_arr, x_zhang_L, x_zhang_R, mtx_L, dist_L, mtx_R, dist_R, (image_size[0],image_size[1]), criteria_stereo, flags)

    return ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat

In [7]:
ret_R_list = []
mtx_R_list = []
ret_L_list = []
mtx_L_list = []
retS_list = []
new_mtxL_list = []
new_mtxR_list = []
Rot_list = []
Trns_list = []
Emat_list = []
Fmat_list = []
P_R_org_list = []
P_L_org_list = []

xrange_0 = np.linspace(-6000,6000,1)
yrange_0 = np.linspace(-4000,4000,1)
zrange_0 = ([20000])

xrange_1 = np.linspace(-6000,6000,2)
yrange_1 = np.linspace(-4000,4000,2)
zrange_1 = ([20000])

xrange_2 = np.linspace(-6000,6000,4)
yrange_2 = np.linspace(-4000,4000,4)
zrange_2 = ([20000])

xrange_3 = np.linspace(-6000,6000,6)
yrange_3 = np.linspace(-4000,4000,6)
zrange_3 = ([20000])

xrange_4 = np.linspace(-6000,6000,8)
yrange_4 = np.linspace(-4000,4000,8)
zrange_4 = ([20000])

xrange_5 = np.linspace(-6000,6000,10)
yrange_5 = np.linspace(-4000,4000,10)
zrange_5 = ([20000])


x_range = ([xrange_0,xrange_1,xrange_2,xrange_3,xrange_4,xrange_5])
y_range = ([yrange_0,yrange_1,yrange_2,yrange_3,yrange_4,yrange_5])
z_range = ([zrange_0,zrange_1,zrange_2,zrange_3,zrange_4,zrange_5])
# x_range = ([xrange_0])
# y_range = ([yrange_0])
# z_range = ([zrange_0])



for i in range(len(z_range)):

    world_pts_arr = 0
    x_zhang_R = 0
    x_zhang_L = 0
    
    world_pts_arr,x_zhang_R,x_zhang_L, P_R, P_L = repeat_func(x_range[i],y_range[i],z_range[i])

    ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat = calib(world_pts_arr,x_zhang_R,x_zhang_L)

    ret_R_list.append([ret_R])
    mtx_R_list.append([mtx_R])
    ret_L_list.append([ret_L])
    mtx_L_list.append([mtx_L])
    retS_list.append([retS])
    new_mtxL_list.append([new_mtxL])
    new_mtxR_list.append([new_mtxR])
    Rot_list.append([Rot])
    Trns_list.append([Trns])
    Emat_list.append([Emat])
    Fmat_list.append([Fmat])
    P_R_org_list.append(P_R)
    P_L_org_list.append(P_L)

(5, 140, 3)
World points:  (5, 1, 140, 3)
Image points:  (5, 140, 2)
(20, 140, 3)
World points:  (20, 1, 140, 3)
Image points:  (20, 140, 2)
(80, 140, 3)
World points:  (80, 1, 140, 3)
Image points:  (80, 140, 2)
(180, 140, 3)
World points:  (180, 1, 140, 3)
Image points:  (180, 140, 2)
(320, 140, 3)
World points:  (320, 1, 140, 3)
Image points:  (320, 140, 2)
(500, 140, 3)
World points:  (500, 1, 140, 3)
Image points:  (500, 140, 2)


In [ ]:
mtx_L_list

[[array([[2.73037611e+03, 0.00000000e+00, 9.67999784e+02],
         [0.00000000e+00, 2.73037607e+03, 6.08000191e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])]]

In [ ]:
P_L_list = []
P_R_list = []

for i in range(len(ret_R_list)):
    R1,R2,P_L_est,P_R_est,Q,roi_left,roi_right = cv2.stereoRectify(new_mtxL_list[i][0], ret_L_list[i][0], new_mtxR_list[i][0], ret_R_list[i][0], (image_size[0],image_size[1]), Rot_list[i][0], Trns_list[i][0],flags = cv2.CALIB_ZERO_DISPARITY)
    P_L_list.append(P_L_est)
    P_R_list.append(P_R_est)

In [8]:
%store P_L_org_list,P_R_org_list,retS_list,new_mtxL_list,ret_L_list,new_mtxR_list,ret_R_list,Rot_list,Trns_list >"C:\Users\PC1\Documents\thesis\thesis_venv\calibration\distances\twentymeters_pixel.txt"

Writing 'P_L_org_list,P_R_org_list,retS_list,new_mtxL_list,ret_L_list,new_mtxR_list,ret_R_list,Rot_list,Trns_list' (tuple) to file 'C:\Users\PC1\Documents\thesis\thesis_venv\calibration\distances\twentymeters_pixel.txt'.
